# Ultra short guide for wandb API data loading

In [1]:
import wandb

# Load data from a run

In [4]:
run_id = "kn9o0vjo"
api = wandb.Api()
run = api.run("philipwastaken/02456_project/runs/" + run_id)
history = run.scan_history()

# Extract specific column

See here for guide https://docs.wandb.ai/guides/track/public-api-guide

In [5]:
mean_train_reward = [row['mean_train_reward'] for row in history]
mean_train_reward

In [9]:
run.summary

{'validate/sims_per_val': 32, 'epsilon': 0.1, '_runtime': 86406.91235685349, '_timestamp': 1669121699.888934, 'model_path': '/zhome/65/7/156117/02456_project/models/scale84_2022_11_21_13DD54DD50D847060_dqnet.pt', 'frame_count': 7329940, 'validate/duration': 20.30483224405907, 'validate/avg_reward': 2230, '_step': 29811, 'start_time': '2022-11-21T13:54:58.640355', 'mean_train_reward': 3390, 'last_recorded_time': '2022-11-22T13:54:59.889928', 'validate/avg_frame_count': 321.1875}